Import some possible required packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.interpolate import interp2d, LinearNDInterpolator
import movies_from_plots as mfp
import selfpropelledparticlevoronoi as sppv
import pathlib
from scipy import ndimage as nd 
import h5py as h5
import importlib

In [ ]:
importlib.reload(mfp)
importlib.reload(sppv)

The model implemented is of the form:

For a center $c_i$

$$F_i = \sum_{v_\mu \in N_i}{f_{\mu i}}$$

For a vertex $v_\mu$
$$F_\mu = \sum_{c_i \in N_\mu}{f_{i\mu}} + \sum_{v_\nu \in N_\mu}{f_{\nu\mu}}$$ 

Where 

$$f_{ab} = -k(r_{ab}-r_0)\hat{n}_{ab}$$

In [ ]:
import runspringnetsimul as rsns
rsns

In [ ]:
import makefiguresandgraphics as mfag

mfag

In [ ]:
import makenetworkmovie as mnwm
mnwm

In [ ]:
import movieforcefieldmovie as mffm
mffm

In [ ]:
filename = input('Video file name (with .mp4 included): ')
img_array,size = mfp.loadframe(0,498,1,'f')
mfp.savevideo(img_array,filename,size,'f')

A model for T1 transition might possibly be:

for 2 vertices $A$ and $B$, if their distance $|\vec{AB}|$ is less than or equal to a threshold value, they change their coordinate positions to: 

$$

(A',B') = \frac{\sqrt{3}}{2}(R_{\pm \pi/2}\vec{AB}+1/2\vec{AB}) 

$$

The transition can be modelled in two different ways:

1 - We describe a simple discrete model where 

$$
P[(A,B)\rightarrow(A',B')]=
\begin{cases}
0, & \text{for } |\vec{AB}| > l_\text{thresh}\\
1,  & \text{for } |\vec{AB}| \leq l_\text{thresh}
\end{cases}
$$

2 - Or we can follow a more continuous, physically inspired approach, where:

$$

P[(A,B)\rightarrow(A',B')] \propto \exp{(-\beta K |\vec{AB}|²)}

$$

3 - A mixed approach could be helpful to ensure the transition occurs at a reasonable rate.

$$
P[(A,B)\rightarrow(A',B')]=
\begin{cases}
0, & \text{for } |\vec{AB}| > l_\text{thresh}\\
\exp{(-\beta K |\vec{AB}|²)},  & \text{for } |\vec{AB}| \leq l_\text{thresh}
\end{cases}
$$

Where $\beta$ is meant to represent thermal energy, or some energy coefficient threshold. 